In [1]:
from transformers import Qwen2VLForConditionalGeneration, Qwen2VLProcessor, BitsAndBytesConfig
from qwen_vl_utils import process_vision_info
import torch

# BitsAndBytesConfig int-4 config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True, bnb_4bit_use_double_quant=True, bnb_4bit_quant_type="nf4", bnb_4bit_compute_dtype=torch.bfloat16
)

# Load model and tokenizer
vl_model = Qwen2VLForConditionalGeneration.from_pretrained(
    "Qwen/Qwen2-VL-7B-Instruct", device_map="auto", torch_dtype=torch.bfloat16, quantization_config=bnb_config
)
vl_model.eval()

`Qwen2VLRotaryEmbedding` can now be fully parameterized by passing the model config through the `config` argument. All other arguments will be removed in v4.46


Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

Qwen2VLForConditionalGeneration(
  (visual): Qwen2VisionTransformerPretrainedModel(
    (patch_embed): PatchEmbed(
      (proj): Conv3d(3, 1280, kernel_size=(2, 14, 14), stride=(2, 14, 14), bias=False)
    )
    (rotary_pos_emb): VisionRotaryEmbedding()
    (blocks): ModuleList(
      (0-31): 32 x Qwen2VLVisionBlock(
        (norm1): LayerNorm((1280,), eps=1e-06, elementwise_affine=True)
        (norm2): LayerNorm((1280,), eps=1e-06, elementwise_affine=True)
        (attn): VisionSdpaAttention(
          (qkv): Linear4bit(in_features=1280, out_features=3840, bias=True)
          (proj): Linear4bit(in_features=1280, out_features=1280, bias=True)
        )
        (mlp): VisionMlp(
          (fc1): Linear4bit(in_features=1280, out_features=5120, bias=True)
          (act): QuickGELUActivation()
          (fc2): Linear4bit(in_features=5120, out_features=1280, bias=True)
        )
      )
    )
    (merger): PatchMerger(
      (ln_q): LayerNorm((1280,), eps=1e-06, elementwise_affine=True)


In [2]:
min_pixels = 448 * 448
max_pixels = 896 * 896 
vl_model_processor = Qwen2VLProcessor.from_pretrained(
    "Qwen/Qwen2-VL-7B-Instruct", min_pixels=min_pixels, max_pixels=max_pixels
)

In [21]:

# Пример текстов для получения эмбендингов
texts = [
    "Это пример текста для извлечения эмбеддингов.",
    "Еще один образец текстового описания."
]

# Предобработка текста
text_inputs = vl_model_processor(text=texts, padding=True, return_tensors="pt").to(vl_model.device)

# Прогон через модель с получением скрытых состояний
with torch.no_grad():
    outputs = vl_model(
        **text_inputs,
        output_hidden_states=True,  # Запрос скрытых состояний
        return_dict=True
    )

# Извлекаем последний скрытый слой
last_hidden_states = outputs.hidden_states[-1]  # [batch_size, seq_length, hidden_dim]

# Маска для токенов (чтобы не учитывать паддинговые токены при вычислении среднего)
attention_mask = text_inputs['attention_mask']
expanded_mask = attention_mask.unsqueeze(-1).expand(last_hidden_states.size()).float()  # [batch_size, seq_length, hidden_dim]

# Вычисляем средний эмбеддинг по непаддинговым токенам
sum_embeddings = torch.sum(last_hidden_states * expanded_mask, dim=1)
sum_mask = torch.clamp(attention_mask.sum(dim=1, keepdim=True), min=1e-9)  # чтобы не делить на ноль
mean_embeddings = sum_embeddings / sum_mask

# mean_embeddings будет размерностью [batch_size, hidden_dim] и может использоваться в RAG для индексации.
mean_embeddings = mean_embeddings.detach().cpu().numpy()

In [6]:
3584

{'input_ids': tensor([[ 92211,  24634, 131616,  70895,   1478,  19849,  23064, 124991,  55757,
          18492,  20928,   6442, 132705,  70338,  15390,   6715,     13],
        [151643, 151643, 151643, 151643, 151643, 151643,  55091, 125103, 129385,
         127273, 126499,  70895, 130928,   8215,  29665,  38180,     13]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [35]:
from PIL import Image
image = Image.open("1.jpg")
inputs = vl_model_processor(images=image, return_tensors="pt")

with torch.no_grad():
    outputs = vl_model.get_image_features(**inputs) 
    # Если метод get_image_features не доступен, смотрите документацию модели.
    # Альтернативно может быть:
    # outputs = model(**inputs)

image_embeddings = outputs

TypeError: argument of type 'NoneType' is not iterable

In [4]:
from PIL import Image
import numpy as np
import base64
import requests
from io import BytesIO

def get_text_embeddings(texts: list[str]) -> np.ndarray:
    # Предобработка текста
    text_inputs = vl_model_processor(text=texts, padding=True, return_tensors="pt").to(vl_model.device)

    with torch.no_grad():
        outputs = vl_model(
            **text_inputs,
            output_hidden_states=True,
            return_dict=True
        )
    last_hidden_states = outputs.hidden_states[-1]  # [batch_size, seq_length, hidden_dim]

    attention_mask = text_inputs['attention_mask']
    expanded_mask = attention_mask.unsqueeze(-1).expand(last_hidden_states.size()).float()
    sum_embeddings = torch.sum(last_hidden_states * expanded_mask, dim=1)
    sum_mask = torch.clamp(attention_mask.sum(dim=1, keepdim=True), min=1e-9)
    mean_embeddings = sum_embeddings / sum_mask
    return mean_embeddings.detach().cpu().numpy()



def get_image_embeddings(image_url: str):
    response = requests.get(image_url)
    image_bytes = response.content
    image = Image.open(BytesIO(image_bytes)).convert("RGB")

    # Определяем формат или используем "jpeg" по умолчанию
    image_format = image.format.lower() if image.format else "jpeg"
    base64_image = base64.b64encode(image_bytes).decode('utf-8')
    data_url = f"data:image/{image_format};base64,{base64_image}"

    messages = [
        {
            "role": "user",
            "content": [
                {
                    "type": "image",
                    "image": data_url
                },
                {
                    "type": "text",
                    "text": "Опишите содержимое этого изображения: <|image|>"
                },
            ],
        }
    ]

    # Генерируем текстовый prompt
    text = vl_model_processor.apply_chat_template(messages, tokenize=False, add_generation_prompt=False)

    # Обрабатываем визуальную информацию
    image_inputs, video_inputs = process_vision_info(messages)

    # Получаем готовые входы для модели
    inputs = vl_model_processor(
        text=[text],
        images=image_inputs,
        videos=video_inputs,
        padding=True,
        return_tensors="pt"
    ).to(vl_model.device)

    with torch.no_grad():
        outputs = vl_model(
            **inputs,
            output_hidden_states=True,
            return_dict=True
        )

    # Последний слой скрытых состояний
    last_hidden_states = outputs.hidden_states[-1]  # [batch_size, seq_length, hidden_dim]

    # Находим токены изображения в input_ids
    image_token_id = vl_model.config.image_token_id
    image_token_mask = (inputs["input_ids"] == image_token_id)  # bool mask для всех позиций токенов изображения

    # Усредняем по всем токенам, соответствующим изображению
    # Если изображение одно, то будет одна группа image-токенов.
    # Если image_token_mask пустой (нет токенов изображения), будет ошибка деления на ноль.
    # Предполагается, что <|image|> есть в тексте.
    image_token_count = image_token_mask.sum(dim=1, keepdim=True).clamp_min(1e-9)
    image_embeddings = (last_hidden_states * image_token_mask.unsqueeze(-1)).sum(dim=1) / image_token_count

    return image_embeddings.detach().cpu().numpy()

for i in range(20):
    # Пример использования
    image_url = "https://i0.wp.com/quorace.com/wp-content/uploads/2016/11/%D0%93%D1%801.jpg"
    image_embeds = get_image_embeddings(image_url)
    print("Эмбеддинги изображения:", image_embeds.shape)
# Теперь у вас есть векторы text_embeds и image_embeds, которые можно использовать в RAG (например, занести в векторное хранилище).

Эмбеддинги изображения: (1, 3584)
Эмбеддинги изображения: (1, 3584)
Эмбеддинги изображения: (1, 3584)
Эмбеддинги изображения: (1, 3584)
Эмбеддинги изображения: (1, 3584)
Эмбеддинги изображения: (1, 3584)
Эмбеддинги изображения: (1, 3584)
Эмбеддинги изображения: (1, 3584)
Эмбеддинги изображения: (1, 3584)
Эмбеддинги изображения: (1, 3584)
Эмбеддинги изображения: (1, 3584)
Эмбеддинги изображения: (1, 3584)
Эмбеддинги изображения: (1, 3584)
Эмбеддинги изображения: (1, 3584)
Эмбеддинги изображения: (1, 3584)
Эмбеддинги изображения: (1, 3584)
Эмбеддинги изображения: (1, 3584)
Эмбеддинги изображения: (1, 3584)
Эмбеддинги изображения: (1, 3584)
Эмбеддинги изображения: (1, 3584)


In [3]:
# Test
messages = [
    {
        "role": "user",
        "content": [
            {
                "type": "image",
                "image": "https://i0.wp.com/quorace.com/wp-content/uploads/2016/11/%D0%93%D1%801.jpg",
            },
            {"type": "text", "text": "Что изображено на фото?"},
        ],
    }
]


text = vl_model_processor.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
)

for i in range(20):
    image_inputs, video_inputs = process_vision_info(messages)
    inputs = vl_model_processor(
        text=[text],
        images=image_inputs,
        videos=video_inputs,
        padding=True,
        return_tensors="pt",
    )
    inputs = inputs.to("cuda")
    
    # Inference: Generation of the output
    generated_ids = vl_model.generate(**inputs, max_new_tokens=128)
    generated_ids_trimmed = [
        out_ids[len(in_ids) :] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
    ]
    output_text = vl_model_processor.batch_decode(
        generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False
    )
    print(output_text)

['На фото изображена диаграмма, представляющая данные в виде столбиков. Диаграмма имеет четыре столбика, каждый из которых соответствует определенному значению. Столбик, соответствующий "плат", имеет самое высокое значение, достигающее 50%. Столбик, соответствующий "3", имеет значение около 10%. Столбик, соответствующий "плат", имеет значение около 20%. Столбик, соответствующий "плат", имеет значение около 5%.']
['На фото изображена диаграмма, представляющая данные в виде столбиков. Диаграмма имеет четыре столбика, каждый из которых соответствует определенному значению. Столбик, соответствующий "плат", имеет самое высокое значение, достигающее 50%. Столбик, соответствующий "3", имеет значение около 10%. Столбик, соответствующий "плат", имеет значение около 20%. Столбик, соответствующий "плат", имеет значение около 5%.']
['На фото изображена диаграмма, представляющая данные в виде столбиков. Диаграмма имеет четыре столбика, каждый из которых соответствует определенному значению. Столбик

In [ ]:
1.43  13